# Clusterização hieráquica no BERT

### Pegar os embeddings

In [1]:
from transformers import BertTokenizer, BertModel

MODEL_PATH = 'bert-base-cased'

tokenizer = BertTokenizer.from_pretrained(MODEL_PATH, do_lower_case=False)

model = BertModel.from_pretrained('bert-base-cased')

for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


embeddings.position_ids 	 torch.Size([1, 512])
embeddings.word_embeddings.weight 	 torch.Size([28996, 768])
embeddings.position_embeddings.weight 	 torch.Size([512, 768])
embeddings.token_type_embeddings.weight 	 torch.Size([2, 768])
embeddings.LayerNorm.weight 	 torch.Size([768])
embeddings.LayerNorm.bias 	 torch.Size([768])
encoder.layer.0.attention.self.query.weight 	 torch.Size([768, 768])
encoder.layer.0.attention.self.query.bias 	 torch.Size([768])
encoder.layer.0.attention.self.key.weight 	 torch.Size([768, 768])
encoder.layer.0.attention.self.key.bias 	 torch.Size([768])
encoder.layer.0.attention.self.value.weight 	 torch.Size([768, 768])
encoder.layer.0.attention.self.value.bias 	 torch.Size([768])
encoder.layer.0.attention.output.dense.weight 	 torch.Size([768, 768])
encoder.layer.0.attention.output.dense.bias 	 torch.Size([768])
encoder.layer.0.attention.output.LayerNorm.weight 	 torch.Size([768])
encoder.layer.0.attention.output.LayerNorm.bias 	 torch.Size([768])
encoder.la

In [23]:
# print(type(md[k]))
# list(tokenizer.vocab.keys())


In [13]:
!pip install torch

import torch
import numpy as np

# MODEL_PATH = 'fagner/envoy'

# md = 

md = torch.load('../../models/NLI/pytorch_model.bin',map_location='cpu')
# md = torch.load('../models/bert/pytorch_model.bin',map_location='cpu')
# md = torch.load('../harena-asm/models/biobert/pytorch_model.bin',map_location='cpu')
# md = torch.load('../models/biobert/pytorch_model.bin',map_location='cpu')
# md = torch.load('../harena-asm/models/bert-large-cased/pytorch_model.bin',map_location='cpu')

# embeds = []
# for k in md:
#     if (k == 'bert.embeddings.word_embeddings.weight'):
#         embeds = md[k]
        
X = np.array(embeds)

words = []
with open('../../models/NLI/vocab.txt', "r") as f:
    words = f.readlines()
    
X_reduzido = X
words_reduzido = words

# Clusterização hierárquica

### Funções pro plot

In [14]:
from matplotlib import pyplot as plt

def plot_clustering(X_red, labels, title=None):
    plt.rcParams['font.sans-serif'] = ['Source Han Sans TW', 'sans-serif']
    x_min, x_max = np.min(X_red, axis=0), np.max(X_red, axis=0)
    X_red = (X_red - x_min) / (x_max - x_min)

    plt.figure(figsize=(6, 4))
    for i in range(X_red.shape[0]):
        plt.text(X_red[i, 0], X_red[i, 1], str(words_reduzido[i]),
                 color=plt.cm.nipy_spectral(labels[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

    plt.xticks([])
    plt.yticks([])
    if title is not None:
        plt.title(title, size=17)
    plt.axis('off')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

In [15]:
from scipy.cluster.hierarchy import dendrogram
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

### Clusterização

In [16]:
from time import time
from sklearn.cluster import AgglomerativeClustering
from sklearn import manifold

clustering = AgglomerativeClustering(linkage='ward', n_clusters=6000)
t0 = time()
model = clustering.fit(X_reduzido)
print("%s :\t%.2fs" % ('ward', time() - t0))

ward :	190.63s


In [17]:
clusters_dict = {}
# print(clusters_dict.items())
for (w, x) in zip(words_reduzido, model.labels_):
    if x in clusters_dict:
        clusters_dict[x].append(w.replace('\n', ''))
    else: clusters_dict[x] = [w.replace('\n', '')]

# for i in clusters_dict:
#     print(str(i) + ": " + str(clusters_dict[i]))

In [1]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

clf = NearestCentroid()
clf.fit(X_reduzido, model.labels_)
print(X_reduzido)

clusters_vector = []


for l in range(len(clf.centroids_)):
    tsv_row = ''
    tsv_row += str(clusters_dict[l]) + '\t'

    vector = clf.centroids_[l]
    
    for m in range(len(vector)):
        tsv_row += str(vector[m].tolist()) + '\t'

    clusters_vector.append(tsv_row)

with open('clustering-results/biobert/base-cased/6000clusters.tsv', "w") as f:
    for e in clusters_vector:
        print (e, file=f) 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.neighbors.nearest_centroid module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


NameError: name 'X_reduzido' is not defined

### plotar dendograma

In [11]:
from sklearn import manifold

X_red = manifold.SpectralEmbedding(n_components=2).fit_transform(X_reduzido)

SystemError: gstrf was called with invalid arguments

In [ ]:
plot_clustering(X_red, clustering.labels_, 'ward')

In [ ]:
plot_dendrogram(model, truncate_mode='level', p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
len(set(model.labels_))